## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df = pd.DataFrame(application_df)
application_df

2024-07-06 16:33:19.126909: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [2]:
!pip install keras-tuner

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
clean_application_df = application_df.drop(['EIN','NAME'], axis=1)

In [4]:
clean_application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [5]:
# Determine the number of unique values in each column.
clean_application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

### For columns with more than 10 unique values, determine the number of data points for each unique value
### Create a new value called 'Other' that contains rare categorical variables

In [6]:
# For columns with more than 10 unique values ('APPLICATION_TYPE' and 'CLASSIFICATION'), determine the number of data points for each unique value
# Create a list of application types to be replaced. Replace rare categorical variables with 'Other'

# Step 1: Determine the number of data points for each unique value
application_type_counts = clean_application_df['APPLICATION_TYPE'].value_counts()

# Step 2: Create a list of application types to be replaced
# Define a threshold for what we consider as 'rare'
threshold = 10
application_types_to_replace = application_type_counts[application_type_counts < threshold].index.tolist()

# Step 3: Replace rare application types with 'Other' using .replace()
clean_application_df['APPLICATION_TYPE'] = clean_application_df['APPLICATION_TYPE'].replace(application_types_to_replace, "Other")

# Step 4: Check to make sure replacement was successful
print(clean_application_df['APPLICATION_TYPE'].value_counts())

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: count, dtype: int64


In [7]:
# Do the same for 'Classification'

# Step 1: Determine the number of data points for each unique value
classification_counts = clean_application_df['CLASSIFICATION'].value_counts()

# Step 2: Create a list of classification types to be replaced
# Define a threshold for what we consider as 'rare'
threshold = 10
classifications_to_replace = classification_counts[classification_counts < threshold].index.tolist()

# Step 3: Replace rare classification types with 'Other' using .replace()
clean_application_df['CLASSIFICATION'] = clean_application_df['CLASSIFICATION'].replace(classifications_to_replace, "Other")

# Step 4: Check to make sure replacement was successful
print(clean_application_df['CLASSIFICATION'].value_counts())

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Other       98
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
Name: count, dtype: int64


In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
cat_application_df = pd.get_dummies(clean_application_df[['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE',
                                      'ORGANIZATION', 'INCOME_AMT',	'SPECIAL_CONSIDERATIONS', 'STATUS', 'IS_SUCCESSFUL', 'ASK_AMT']])

In [9]:
cat_application_df

,STATUS,IS_SUCCESSFUL,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,1,5000,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,1,108590,False,False,False,False,False,False,True,...,True,False,False,False,False,False,False,False,True,False
2,1,0,5000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,1,1,6692,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,True,False
4,1,1,142590,False,False,False,False,False,False,True,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,0,5000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34295,1,0,5000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34296,1,0,5000,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,True,False
34297,1,1,5000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [10]:
# Split our preprocessed data into our features and target arrays
# Remove "IS_SUCCESSFUL" target from features data
y = cat_application_df.IS_SUCCESSFUL.values
X = cat_application_df.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Compile, Train and Evaluate the Model -- adjust number of hidden layers and epochs to optimize the model

In [19]:
# Define the model - deep neural net
nn_model = tf.keras.models.Sequential()

# Add first Dense layer, including the input layer (input_dim should match the number of features)
input_dim = X_train_scaled.shape[1]
nn_model.add(tf.keras.layers.Dense(units=32, activation="relu", input_dim=input_dim))

# Add additional hidden layers
nn_model.add(tf.keras.layers.Dense(units=24, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Add the output layer (assuming binary classification)
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model.summary()

# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=20)

# Evaluate the model
loss, accuracy = nn_model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {accuracy}")

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_28 (Dense)                │ (None, 32)             │         2,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 24)             │           792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 8)              │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305 (12.91 KB)

 Trainable params: 3,305 (12.91 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7038 - loss: 0.6034
Epoch 2/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7352 - loss: 0.5479
Epoch 3/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7341 - loss: 0.5479
Epoch 4/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7369 - loss: 0.5447
Epoch 5/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7354 - loss: 0.5452
Epoch 6/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7356 - loss: 0.5454
Epoch 7/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7390 - loss: 0.5423
Epoch 8/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7400 - loss: 0.5418
Epoch 9/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7389 - loss: 0.5403
Epoch 10/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7412 - loss: 0.5395
Epoch 11/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7365 - loss: 0.5406
Epoch 12/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

### Compile, Train and Evaluate the Model -- try a different optimizer

In [16]:
# Define the model - deep neural net
nn_model = tf.keras.models.Sequential()

# Add first Dense layer, including the input layer (input_dim should match the number of features)
input_dim = X_train_scaled.shape[1]
nn_model.add(tf.keras.layers.Dense(units=32, activation="relu", input_dim=input_dim))

# Add additional hidden layers
nn_model.add(tf.keras.layers.Dense(units=24, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Add the output layer (assuming binary classification)
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model.summary()

# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=40)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                │ (None, 32)             │         2,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 24)             │           792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 8)              │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305 (12.91 KB)

 Trainable params: 3,305 (12.91 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6713 - loss: 0.6162
Epoch 2/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7254 - loss: 0.5576
Epoch 3/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7319 - loss: 0.5527
Epoch 4/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7316 - loss: 0.5520
Epoch 5/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7273 - loss: 0.5550
Epoch 6/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7362 - loss: 0.5460
Epoch 7/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7422 - loss: 0.5397
Epoch 8/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7381 - loss: 0.5427
Epoch 9/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7389 - loss: 0.5428
Epoch 10/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7322 - loss: 0.5475
Epoch 11/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7417 - loss: 0.5380
Epoch 12/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step

### Since the RMS Prop optimizer helped slightly, try another optimizer to increase the overall accuracy score

In [24]:
# Define the model - deep neural net
nn_model = tf.keras.models.Sequential()

# Add first Dense layer, including the input layer (input_dim should match the number of features)
input_dim = X_train_scaled.shape[1]
nn_model.add(tf.keras.layers.Dense(units=32, activation="relu", input_dim=input_dim))

# Add additional hidden layers
nn_model.add(tf.keras.layers.Dense(units=24, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Add the output layer (assuming binary classification)
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model.summary()

# Compile the model
optimizer = tf.keras.optimizers.Adamax(learning_rate=0.002)
nn_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=40)

# Evaluate the model
loss, accuracy = nn_model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {accuracy}")

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_48 (Dense)                │ (None, 32)             │         2,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 24)             │           792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 8)              │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_51 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305 (12.91 KB)

 Trainable params: 3,305 (12.91 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6886 - loss: 0.6213
Epoch 2/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7284 - loss: 0.5599
Epoch 3/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7360 - loss: 0.5505
Epoch 4/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7379 - loss: 0.5438
Epoch 5/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7385 - loss: 0.5423
Epoch 6/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7368 - loss: 0.5441
Epoch 7/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7380 - loss: 0.5403
Epoch 8/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7386 - loss: 0.5429
Epoch 9/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7350 - loss: 0.5446
Epoch 10/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7361 - loss: 0.5445
Epoch 11/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7359 - loss: 0.5394
Epoch 12/40
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

### Try again with more layers and more neurons

In [25]:
# Add layers with more neurons
input_dim = X_train_scaled.shape[1]
nn_model.add(tf.keras.layers.Dense(units=64, activation="relu", input_dim=input_dim))  # Increased neurons
nn_model.add(tf.keras.layers.Dense(units=32, activation="relu"))  # Increased neurons
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))  # Increased neurons
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model with RMSprop optimizer
nn_model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model
loss, accuracy = nn_model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {accuracy}")

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7059 - loss: 0.5933
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7374 - loss: 0.5481
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7352 - loss: 0.5457
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7368 - loss: 0.5482
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7368 - loss: 0.5450
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7359 - loss: 0.5475
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7380 - loss: 0.5454
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7417 - loss: 0.5432
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7391 - loss: 0.5426
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7427 - loss: 0.5429
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7369 - loss: 0.5472
Epoch 12/100
804/804 ━━━━━━━━━━━━━━━━━━━━

### Try again with dropout layers and a different learning rate

In [26]:
# Add layers with more neurons and additional techniques
input_dim = X_train_scaled.shape[1]
nn_model.add(tf.keras.layers.Dense(units=128, activation="relu", input_dim=input_dim))
nn_model.add(tf.keras.layers.Dropout(0.5))  # Add dropout layer
nn_model.add(tf.keras.layers.Dense(units=64, activation="relu"))
nn_model.add(tf.keras.layers.Dropout(0.5))  # Add dropout layer
nn_model.add(tf.keras.layers.Dense(units=32, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model with RMSprop optimizer and different learning rate
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.003)
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model with different batch size and epochs
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100, batch_size=64, validation_split=0.2)

# Evaluate the model
loss, accuracy = nn_model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {accuracy}")

Epoch 1/100
322/322 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.7035 - loss: 0.6074 - val_accuracy: 0.7376 - val_loss: 0.5512
Epoch 2/100
322/322 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7424 - loss: 0.5481 - val_accuracy: 0.7308 - val_loss: 0.5529
Epoch 3/100
322/322 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7379 - loss: 0.5461 - val_accuracy: 0.7322 - val_loss: 0.5491
Epoch 4/100
322/322 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7342 - loss: 0.5514 - val_accuracy: 0.7357 - val_loss: 0.5518
Epoch 5/100
322/322 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7404 - loss: 0.5440 - val_accuracy: 0.7355 - val_loss: 0.5551
Epoch 6/100
322/322 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7361 - loss: 0.5425 - val_accuracy: 0.7343 - val_loss: 0.5539
Epoch 7/100
322/322 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7407 - loss: 0.5405 - val_accuracy: 0.7310 - val_loss: 0.5575
Epoch 8/100
322/322 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7431 - loss: 0.5405 - val_accu

In [27]:
# Save the model to HDF5 file
nn_model.save("AlphabetSoupCharity_Optimization2.h5")